# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import utils
import datahandler

import numpy as np
import pandas as pd
from tqdm import tqdm
import osmnx
import osmnx.utils_graph
import networkx
import math
import osmnx.distance
import geopandas as gpd
import shapely.geometry


# Load dataset

In [4]:
dataset_id = "oslo"

data_preprocessor = datahandler.DataPreprocessorOUS(dataset_id)
data_preprocessor.execute()

data_loader = datahandler.DataLoader(dataset_id)
data_loader.execute()


Loading dataset: 100%|██████████| 6/6 [00:13<00:00,  2.26s/it]


# Main

In [5]:
def get_centroid_max_distance():
    akershus_gdf: gpd.GeoDataFrame = gpd.read_file(os.path.join(constants.PROJECT_DIRECTORY_PATH, "data", "ssb_2019_akershus_polygon_epsg4326.geojson"))
    oslo_gdf: gpd.GeoDataFrame = gpd.read_file(os.path.join(constants.PROJECT_DIRECTORY_PATH, "data", "ssb_2019_oslo_polygon_epsg4326.geojson"))

    akershus_gdf['dissolve_field'] = 'Region'
    oslo_gdf['dissolve_field'] = 'Region'
    combined_gdf: gpd.GeoDataFrame = pd.concat([akershus_gdf, oslo_gdf], ignore_index=True)
    gdf: gpd.GeoDataFrame = combined_gdf.dissolve(by='dissolve_field')

    gdf = gdf.to_crs(epsg=32633)
    centroid: shapely.geometry.Point = gdf.geometry.centroid.iloc[0]

    lon, lat = utils.utm_to_geographic(centroid.x, centroid.y)

    max_distance = gdf.geometry.bounds.apply(
        lambda row: max(
            centroid.distance(shapely.geometry.Point(row['minx'], row['miny'])),
            centroid.distance(shapely.geometry.Point(row['minx'], row['maxy'])),
            centroid.distance(shapely.geometry.Point(row['maxx'], row['miny'])),
            centroid.distance(shapely.geometry.Point(row['maxx'], row['maxy']))
        ),
        axis=1
    ).max()

    return (lat, lon), max_distance


In [6]:
def get_graph(centroid, distance, epsg: str = f"EPSG:326{33}"):
    graph = osmnx.graph_from_point(
        centroid,
        distance,
        dist_type="bbox",
        network_type="drive",
        simplify=True,
        retain_all=False,
        truncate_by_edge=False
    )
    graph = osmnx.project_graph(graph, to_crs=epsg)

    return graph


In [7]:
def set_graph_weights(graph: networkx.MultiDiGraph):
    speeds_normal = {
        30: 26.9,
        40: 45.4,
        50: 67.6,
        60: 85.8,
        70: 91.8,
        80: 104.2,
        90: 112.1,
        100: 120.0,
        110: 128.45,
        120: 136.9
    }
    INTERSECTION_PENALTY = 10

    for u, v, data in graph.edges(data=True):
        if "maxspeed" in data and data["maxspeed"] != "NO:urban":
            if isinstance(data["maxspeed"], list):
                speed_limits = [speeds_normal.get(int(s), int(s)) for s in data["maxspeed"]]
                speed_limit = sum(speed_limits) / len(speed_limits)
            else:
                speed_limit = speeds_normal.get(int(data["maxspeed"]), int(data["maxspeed"]))
            
            avg_speed = speed_limit * 0.65
        else:
            avg_speed = 50
        
        data["time"] = data["length"] / (avg_speed * 1000/60)

        if "junction" in graph.nodes[u] or "highway" in graph.nodes[u] and graph.nodes[u]["highway"] == "traffic_signals":
            data["time"] += INTERSECTION_PENALTY / 60
        if "junction" in graph.nodes[v] or "highway" in graph.nodes[v] and graph.nodes[v]["highway"] == "traffic_signals":
            data["time"] += INTERSECTION_PENALTY / 60


In [8]:
def plot_route(graph: networkx.MultiDiGraph, grid_id1, grid_id2):
    x1, y1 = utils.id_to_utm(grid_id1)
    x2, y2 = utils.id_to_utm(grid_id2)

    node1 = osmnx.distance.nearest_nodes(graph, x1, y1)
    node2 = osmnx.distance.nearest_nodes(graph, x2, y2)

    if networkx.has_path(graph, node1, node2):
        shortest_path = osmnx.shortest_path(graph, node1, node2, weight='time')
        osmnx.plot_graph_route(graph, shortest_path)
    else:
        print("No path between the specified nodes.")


In [9]:
def get_node(graph: networkx.MultiDiGraph, node_validator, grid_id, cache):
    if grid_id in cache:
        return cache[grid_id]
    
    x, y = utils.id_to_utm(grid_id)

    while True:
        node = osmnx.distance.nearest_nodes(graph, x, y)
        if networkx.has_path(graph, node, node_validator) and networkx.has_path(graph, node_validator, node):
            cache[grid_id] = node
            return node
        else:
            graph.remove_node(node)


In [11]:
def simplify_graph(graph: networkx.MultiDiGraph, grid_id_validator, grid_ids, cache):
    """Does not work"""
    x, y = utils.id_to_utm(grid_id_validator)
    node_validator = osmnx.distance.nearest_nodes(graph, x, y)

    edges = []
    for grid_id in tqdm(grid_ids, desc="Simplifying graph"):
        edges.extend(list(graph.edges(get_node(graph, node_validator, grid_id, cache), keys=True)))
    
    return graph.edge_subgraph(edges).copy()


In [12]:
grid_ids = pd.concat([data_loader.enhanced_incidents_df["grid_id"], data_loader.enhanced_depots_df["grid_id"]])
grid_ids = grid_ids.unique()
grid_ids.sort()

centroid, distance = get_centroid_max_distance()
graph = get_graph(centroid, distance)
set_graph_weights(graph)

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
node_validator = osmnx.distance.nearest_nodes(graph, central_depot_x, central_depot_y)
node_cache = {}


Simplifying graph: 100%|██████████| 2616/2616 [25:58<00:00,  1.68it/s]
